In [ ]:
import os
from llama_index.llms.vllm import Vllm
from llama_index.core import Settings
# mistralai/Mistral-Small-Instruct-2409
# tiiuae/Falcon3-7B-Instruct

llm = Vllm(
    model="mistralai/Mistral-Small-Instruct-2409",
    vllm_kwargs={
        "tokenizer_mode": "auto",
        "load_format": "auto",
        "quantization": None,
        },
)

Settings.llm = llm

In [ ]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tools = McpToolSpec(client=mcp_client) # you can also pass list of allowed tools

tools = await mcp_tools.to_tool_list_async()
for tool in tools:
    print(f"{tool.metadata.name}: {tool.metadata.description}")

In [ ]:
async def get_agent(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = FunctionAgent(
        name="Agent",
        description="An agent that can work with our math MCP server.",
        tools=tools,
        llm=Settings.llm,
        system_prompt=SYSTEM_PROMPT,
    )
    return agent


async def handle_user_message(
    message_content: str,
    agent: FunctionAgent,
    agent_context: Context,
    verbose: bool = False,
):
    handler = agent.run(message_content, ctx=agent_context)
    async for event in handler.stream_events():
        if verbose and type(event) == ToolCall:
            print(f"Calling tool {event.tool_name} with kwargs {event.tool_kwargs}")
        elif verbose and type(event) == ToolCallResult:
            print(f"Tool {event.tool_name} returned {event.tool_output}")

    response = await handler
    return str(response)


mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tools = McpToolSpec(client=mcp_client) # you can also pass list of allowed tools

# get the agent
agent = await get_agent(mcp_tools)

# create the agent context
agent_context = Context(agent)


while True:
    user_input = input("Enter your message: ")
    if user_input == "exit":
        break
    print("User: ", user_input)
    response = await handle_user_message(user_input, agent, agent_context, verbose=True)
    print("Agent: ", response)
